In [1]:
import numpy as np
import os
import datetime
import pandas as pd
import random
from scipy import stats
# from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from sklearn import preprocessing
from sklearn import datasets
from sklearn import model_selection

In [28]:
from keras import models
from keras import layers
from keras import datasets as kDatasets
from keras import preprocessing as kpreprocess
from keras.utils import np_utils

### Pre-process
- Important to preprocess as all fetures combined, mean of 0 and std of 1 is usually good

In [3]:
features = np.array([[-100.1,3240.1], [-200.2, -234.1], [5000.5, 150.1], [6000.6, -125.1], [9000.9, -673.1]])
scaler = preprocessing.StandardScaler()
features_std = scaler.fit_transform(features)
features_std

array([[-1.12541308,  1.96429418],
       [-1.15329466, -0.50068741],
       [ 0.29529406, -0.22809346],
       [ 0.57385917, -0.42335076],
       [ 1.40955451, -0.81216255]])

### Design Network

In [4]:
network = models.Sequential()
# Fully connected with ReLU activation with 10 Features output 16
network.add(layers.Dense(units=16, activation='relu', input_shape=(10, )))
# Fully connected with ReLU activation
network.add(layers.Dense(units=16, activation='relu'))
# Fully connected with Sigmoid output 1
network.add(layers.Dense(units=1, activation='sigmoid'))
# Compile and add loss function, optimization, and performance metric
network.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


Output Layer Activiation:
- Binary Classification: sigmoid activation function
- Multiclass Classification: k units (# target classes) and softmax activation function
- Regression: one unit with no activiation

Loss Functions:
- Binary Classification: binary_crossentropy
- Multiclass Classification: categorical_crossentropy
- Regression: Mean Square Error

- Opimizer strategy to find parameter functions: stochastic gradient descent (with momentum), adaptive moment estimation
- Metrics for evaluating performance

- https://keras.io/api/losses/probabilistic_losses/#binarycrossentropy-class
- https://en.wikipedia.org/wiki/Loss_functions_for_classification

### Binary Classifier

In [5]:
np.random.seed(0)
num_features = 1000
# Load data
(data_train, target_train), (data_test, target_test) = kDatasets.imdb.load_data(num_words=num_features)

In [6]:
# convert movie review to one hot
tokenizer = kpreprocess.text.Tokenizer(num_words=num_features)
features_train = tokenizer.sequences_to_matrix(data_train, mode='binary')
features_test = tokenizer.sequences_to_matrix(data_test, mode='binary')

In [8]:
# Neural net
network = models.Sequential()
network.add(layers.Dense(units=16, activation='relu', input_shape=(num_features, )))
network.add(layers.Dense(units=16, activation='relu'))
network.add(layers.Dense(units=1, activation='sigmoid'))
network.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [10]:
# Train
history = network.fit(features_train, target_train, epochs=3, verbose=1, batch_size=100,
                      validation_data=(features_test, target_test))

Instructions for updating:
Use tf.cast instead.
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 3s 103us/step - loss: 0.4161 - acc: 0.8134 - val_loss: 0.3347 - val_acc: 0.8586
Epoch 2/3
25000/25000 [==============================] - 2s 64us/step - loss: 0.3250 - acc: 0.8635 - val_loss: 0.3302 - val_acc: 0.8600
Epoch 3/3
25000/25000 [==============================] - 2s 63us/step - loss: 0.3160 - acc: 0.8664 - val_loss: 0.3306 - val_acc: 0.8600


In [13]:
history.history

{'val_loss': [0.33474202746152876, 0.33022729587554933, 0.33062036979198456],
 'val_acc': [0.8586399981975555, 0.8599599974155426, 0.8600000011920929],
 'loss': [0.41608748406171797, 0.3249572507739067, 0.3160091818571091],
 'acc': [0.8133999975919723, 0.8635199983119964, 0.8663999998569488]}

### MultiClass Classifier

In [14]:
np.random.seed(0)
num_features = 5000
# Load data
(data_train, target_train), (data_test, target_test) = kDatasets.reuters.load_data(num_words=num_features)

2113536/2110848 [==============================] - 2s 1us/step


In [15]:
# convert movie review to one hot
tokenizer = kpreprocess.text.Tokenizer(num_words=num_features)
features_train = tokenizer.sequences_to_matrix(data_train, mode='binary')
features_test = tokenizer.sequences_to_matrix(data_test, mode='binary')

In [29]:
# one hot encode target vector to create a target matrix
tar_train = np_utils.to_categorical(target_train)
tar_test = np_utils.to_categorical(target_test)

In [25]:
features_train[0][:10], [x for x in data_train[0] if x < 11]

(array([0., 1., 1., 0., 1., 1., 1., 1., 1., 1.]),
 [1, 2, 2, 8, 10, 5, 5, 7, 5, 6, 7, 4, 8, 6, 6, 7, 9, 6, 6, 8, 6])

In [27]:
max(target_train)

45

In [30]:
# Neural net
network = models.Sequential()
network.add(layers.Dense(units=100, activation='relu', input_shape=(num_features, )))
network.add(layers.Dense(units=100, activation='relu'))
network.add(layers.Dense(units=46, activation='softmax'))
network.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [31]:
# Train
history = network.fit(features_train, tar_train, epochs=3, verbose=1, batch_size=100,
                      validation_data=(features_test, tar_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/3
8982/8982 [==============================] - 4s 490us/step - loss: 1.5621 - acc: 0.6653 - val_loss: 1.1439 - val_acc: 0.7395
Epoch 2/3
8982/8982 [==============================] - 4s 435us/step - loss: 0.8139 - acc: 0.8177 - val_loss: 0.9453 - val_acc: 0.7854
Epoch 3/3
8982/8982 [==============================] - 4s 398us/step - loss: 0.5242 - acc: 0.8844 - val_loss: 0.8891 - val_acc: 0.7970


### Train a Regressor

In [32]:
features, target = datasets.make_regression(n_samples=10000, n_features=3, n_informative=3, n_targets=1,
                                           noise=0.0, random_state=0)
# Divide
features_train, features_test, target_train, target_test = model_selection.train_test_split(features, target,
                                                                                           test_size=0.33, random_state=0)

In [33]:
# Neural net
network = models.Sequential()
network.add(layers.Dense(units=32, activation='relu', input_shape=(features_train.shape[1], )))
network.add(layers.Dense(units=32, activation='relu'))
network.add(layers.Dense(units=1))
network.compile(loss='mse', optimizer='RMSprop', metrics=['mse'])

In [34]:
# Train
history = network.fit(features_train, target_train, epochs=10, verbose=1, batch_size=100,
                      validation_data=(features_test, target_test))

Train on 6700 samples, validate on 3300 samples
Epoch 1/10
6700/6700 [==============================] - 1s 81us/step - loss: 17308.3502 - mean_squared_error: 17308.3502 - val_loss: 17665.7711 - val_mean_squared_error: 17665.7711
Epoch 2/10
6700/6700 [==============================] - 0s 50us/step - loss: 16427.9816 - mean_squared_error: 16427.9816 - val_loss: 16364.2492 - val_mean_squared_error: 16364.2492
Epoch 3/10
6700/6700 [==============================] - 0s 55us/step - loss: 14751.9711 - mean_squared_error: 14751.9711 - val_loss: 14172.1044 - val_mean_squared_error: 14172.1044
Epoch 4/10
6700/6700 [==============================] - 0s 25us/step - loss: 12224.2324 - mean_squared_error: 12224.2324 - val_loss: 11115.3894 - val_mean_squared_error: 11115.3894
Epoch 5/10
6700/6700 [==============================] - 0s 39us/step - loss: 8940.6426 - mean_squared_error: 8940.6426 - val_loss: 7477.5662 - val_mean_squared_error: 7477.5662
Epoch 6/10
6700/6700 [=============================

### Make Predictions

In [35]:
np.random.seed(0)
num_features = 1000
# Load data
(data_train, target_train), (data_test, target_test) = kDatasets.imdb.load_data(num_words=num_features)

In [36]:
# convert movie review to one hot
tokenizer = kpreprocess.text.Tokenizer(num_words=num_features)
features_train = tokenizer.sequences_to_matrix(data_train, mode='binary')
features_test = tokenizer.sequences_to_matrix(data_test, mode='binary')

In [37]:
# Neural net
network = models.Sequential()
network.add(layers.Dense(units=16, activation='relu', input_shape=(num_features, )))
network.add(layers.Dense(units=16, activation='relu'))
network.add(layers.Dense(units=1, activation='sigmoid'))
network.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [39]:
# Train
history = network.fit(features_train, target_train, epochs=3, verbose=1, batch_size=100,
                      validation_data=(features_test, target_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 3s 106us/step - loss: 0.4163 - acc: 0.8131 - val_loss: 0.3343 - val_acc: 0.8585
Epoch 2/3
25000/25000 [==============================] - 2s 72us/step - loss: 0.3248 - acc: 0.8636 - val_loss: 0.3293 - val_acc: 0.8603
Epoch 3/3
25000/25000 [==============================] - 2s 72us/step - loss: 0.3150 - acc: 0.8664 - val_loss: 0.3295 - val_acc: 0.8599


In [41]:
predicted_target = network.predict(features_test)

In [42]:
predicted_target[:10], target_test[:10]

(array([[0.24883738],
        [0.9984263 ],
        [0.50243497],
        [0.77616036],
        [0.9270196 ],
        [0.78844833],
        [0.9846772 ],
        [0.03175363],
        [0.9408301 ],
        [0.9189039 ]], dtype=float32),
 array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1], dtype=int64))